Nicolas, Tess, Abdoulaye


In [16]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Données

## Chargement des données 

In [17]:
df = pd.read_csv("data.csv")
df.head()


,Document,Annotateur,Texte,Début,Fin,Layer,CUI,Type,Confiance,Relation,Source,Cible
0,common_27449845.txt,AU90360,Zirconia Abutments,21.0,39.0,Medical,C0545406,anatomy,high,NaN,NaN,NaN
1,common_27449845.txt,AU90360,zirconia abutments,124.0,142.0,Medical,C0545406,anatomy,high,NaN,NaN,NaN
2,common_27449845.txt,AV45040,Zirconia,21.0,29.0,Medical,A1.4.1.1.1,pharmacologic_substance,high,NaN,NaN,NaN
3,common_27449845.txt,AV45040,single-tooth,165.0,177.0,Medical,A1.2,anatomy,high,NaN,NaN,NaN
4,common_27706165.txt,AU90360,Mycobacterium leprae,109.0,129.0,Medical,C0026922,bacteria_virus,high,NaN,NaN,NaN


In [38]:

df_exploded = pd.read_csv("data_exploded.csv")
df_exploded

,Document,Annotateur,Texte,Début,Fin,Layer,CUI,Type,Confiance,Relation,Source,Cible
0,common_27449845.txt,AU90360,Zirconia,21.0,39.0,Medical,C0545406,anatomy,high,NaN,NaN,NaN
1,common_27449845.txt,AU90360,Abutments,21.0,39.0,Medical,C0545406,anatomy,high,NaN,NaN,NaN
2,common_27449845.txt,AU90360,zirconia,124.0,142.0,Medical,C0545406,anatomy,high,NaN,NaN,NaN
3,common_27449845.txt,AU90360,abutments,124.0,142.0,Medical,C0545406,anatomy,high,NaN,NaN,NaN
4,common_27449845.txt,AV45040,Zirconia,21.0,29.0,Medical,A1.4.1.1.1,pharmacologic_substance,high,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
868,common_28490813.txt,CURATION_USER,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Causes,IFN-γ,secretion
869,common_28490813.txt,CURATION_USER,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Causes,secretion,cell anergy
870,common_28490813.txt,CURATION_USER,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Refers_to,MM,multiple myeloma
871,common_28490813.txt,CURATION_USER,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Refers_to,iNKT,invariant natural killer T


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 610 entries, 0 to 609
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Document    610 non-null    object 
 1   Annotateur  610 non-null    object 
 2   Texte       554 non-null    object 
 3   Début       554 non-null    float64
 4   Fin         554 non-null    float64
 5   Layer       554 non-null    object 
 6   CUI         441 non-null    object 
 7   Type        547 non-null    object 
 8   Confiance   228 non-null    object 
 9   Relation    56 non-null     object 
 10  Source      56 non-null     object 
 11  Cible       56 non-null     object 
dtypes: float64(2), object(10)
memory usage: 57.3+ KB


# Analysis of the results 

## Confidence analysis 

In [19]:


effectifs_confiance = pd.crosstab(
    index=df['Annotateur'],  # Lignes = utilisateurs
    columns=df['Confiance'].fillna("none")  # Colonnes = valeurs de confiance
)
effectifs_confiance


Confiance,high,low,none
Annotateur,,,
AU90360,84,11,32
AV00440,46,16,40
AV45040,58,11,28
CURATION_USER,2,0,282


## Category annotation analysis

In [ ]:
df_category = df_exploded[df_exploded['Texte'].notna()][['Texte', 'Annotateur', 'Type']]

# Ajouter un identifiant unique pour chaque ligne de texte dupliqué
df_category['Texte_ID'] = df_category.groupby('Texte').cumcount()

# Pivot pour transformer les annotateurs en colonnes
df_pivot = df_category.pivot_table(index=['Texte', 'Texte_ID'], columns='Annotateur', values='Type', aggfunc=lambda x: ', '.join(map(str, x)))

# Remettre 'Texte' comme colonne et supprimer l'index additionnel
df_pivot = df_pivot.reset_index().drop(columns=['Texte_ID'])

# Filtrer les textes de ponctuation avec RegEx (les textes ne contenant que des caractères spéciaux)
df_pivot = df_pivot[~df_pivot['Texte'].str.match(r'^[\W_]+$')]
df_pivot 

Annotateur,Texte,AU90360,AV00440,AV45040,CURATION_USER
0,(CYP450),NaN,NaN,anatomy,NaN
8,Abutments,anatomy,NaN,NaN,NaN
9,Abutments,NaN,NaN,NaN,anatomy
10,Activator,physiologic_function,NaN,NaN,NaN
11,Agoraphobia,pathology,NaN,NaN,NaN
...,...,...,...,...,...
812,virus,NaN,NaN,NaN,bacteria_virus
813,virus,NaN,NaN,NaN,bacteria_virus
814,virus,NaN,NaN,NaN,reference_object
815,zirconia,anatomy,NaN,NaN,NaN


In [ ]:
df_pivot_grouped = df_pivot.groupby('Texte').agg(lambda x: ', '.join(x.dropna().astype(str))).reset_index()



Annotateur,Texte,AU90360,AV00440,AV45040,CURATION_USER
3,Agoraphobia,pathology,pathology,pathology,pathology
49,Schizophrenia,pathology,pathology,pathology,pathology
139,infections,pathology,pathology,pathology,pathology


## Group By Annotator 

In [ ]:
annotators = df['Annotateur'].unique()
print(annotators)

['AU90360' 'AV45040' 'AV00440' 'CURATION_USER']


In [10]:
df_AU90360 = df[df['Annotateur'] == 'AU90360']
df_AU90360 = df_AU90360.drop(columns=['Annotateur'])

df_AV45040 = df[df['Annotateur'] == 'AV45040']
df_AV45040 = df_AV45040.drop(columns=['Annotateur'])

df_AV00440 = df[df['Annotateur'] == 'AV00440']
df_AV00440 = df_AV00440.drop(columns=['Annotateur'])

df_CURATION = df[df['Annotateur'] == 'CURATION_USER']
df_CURATION = df_CURATION.drop(columns=['Annotateur'])

## Group By Layer

In [ ]:
df_medical = df_exploded[]      